# Search BDTNS by Sign
Goal of this notebook is to search the [BDTNS](http://bdtns.filol.csic.es) data by signs, irrespective of their reading. For instance, the sign NE may be read bi₂, ne, izi, šeŋ₆, kum₂, lam₂, zah₂, etc. It is easy to search for transliteration (and/or metadata) in the [BDTNS](http://bdtns.filol.csic.es) search page, but there is currently no way to search for a sequence of signs. This is useful, in particular, in two situations. 

1. Sumerological transliteration conventions may differ quite substantially between different schools. Thus, lu₂ kin-gi₄-a, {lu₂}kin-gi₄-a, lu₂ kiŋ₂-gi₄-a and {lu₂}kiŋ₂-gi₄-a all represent the same sequence of signs and the same word (meaning 'messenger'), but without knowledge of the particular set of conventions used it may be difficult to guess which search will yield the desired results. In the sign search one may enter sign readings according to any convention recognized by the ORACC Global Sign List ([OGSL](http://oracc.org/ogsl).

2. In some cases the correct reading and interpretation of a sign sequence may be ambiguous and the ambiguiuty may have been resolved in different ways throughout the database. The names lugal-mudra₅, lugal-suluhi₂ and lugal-siki-su₁₃ all represent the same sign sequence. Which of these is correct is not entirely clear (although the third seems unlikely) and, depending on the research question, may even be unimportant (for instance for an SNA analysis). In the sign search one may enter any of these forms and the results will include all of them.

In [3]:
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()
import re
import pickle

The sign search uses the [OGSL](http://oracc.org/ogsl) data, which are available in JSON format at http://build-oracc.museum.upenn.edu/json/ogsl. The current notebook uses a dataframe that was produced and pickled elsewhere from the JSON data. The dataframe has four columns: 'hex' (the hexagesimal code point of the sign in Unicode), 'name' (the sign name in OGSL), 'utf8' (the UTF8 representation of the sign in cuneiform Unicode block), and 'value' (a particular reading of the sign). The field 'value' is unique for each row; the same sign (represented by 'hex', 'name', and 'utf8' may be repeated as many tiumes as needed to accomodate all readings of that sign. 

In [4]:
with open("../test fasttext/output/ogsl.p", "rb") as f:
    o = pickle.load(f)

The keys of the dictionary `d2` represent all readings of cuneiform signs. The value is the corresponding sign name. 

In [5]:
d2 = dict(zip(o['value'], o['name']))

We can now open the dataframe with the [BDTNS](http://bdtns.filol.csic.es) transliterations. This dataframe was pickled in notebook [2_4_Data_Acquisition_BDTNS.ipnb]. The dataframe has five fields: `id_text` (the [BDTNS](http://bdtns.filol.csic.es) number of a document), `id_line` (a continuouys line numbering that starts at 1 for each new document; integer), `line_label` (the regular [BDTNS](http://bdtns.filol.csic.es) line number), `text` (the transliteration oif the line) and `comments` (any comments added to the line in [BDTNS](http://bdtns.filol.csic.es)).

In [7]:
with open('output/bdtns.p', 'rb') as f: 
    df = pickle.load(f)

In order to separate signs form each other in the transliteration column (`text`) and to ignore elements such as question marks or (half-) brackets we define different types of separators, operators, and flags that may be present in the text or in the sign name. The most common separators are space and hyphen. Curly brackets are placed around determinatives (semantic classifiers, for instance in {d}En-lil₂). Curly brackets and hyphens will be replaced by spaces. The separators in `separators2` are used in compound signs, as in |SI.A|, or |ŠU+NIGIN|. Operators, finally, are also used in compound signs and indicate how the signs are written in relation to each other (on top of each other, one inside the other, etc.). Compound signs that represent a sequence of simple signs (|SI.A| for **dirig** or |A.TU.GAB.LIŠ| for **asal₂**( will be decomposed in their component signs. Compound signs of the type |KA×GAR| for **gu₇**) are not analyzed, but their component parts are aligned with OGSL practices (that is |KA×NINDA| will be re-written as |KA×GAR|, because in OGSL GAR is the name of the sign that can be read ninda or gar).

Finally the flags include various characters that may appear in the transliteration but will be ignored in the search. A search for `ninda`, therefore, will find `ninda`, `[nin]da`, `ninda?`, etc., as well as `gar`, `⸢gar⸣`, `gar!`, etc. (but not `nagar`, see below).

The variable `table` represents a table in which each character in `flags` corresponds to `None`. This is used by the `translate()` method; see below.

In [ ]:
separators = ['{', '}', '-']
separators2 = ['.', '+', '|']
operators = ['&', '%', '@', '×']
flags = "][!?<>⸢⸣⌈⌉*/"
table = str.maketrans(dict.fromkeys(flags))

In [ ]:
def signs(row):  
    row_l = []
    sign_names = []
    sign_sequence = ''
    row = row.translate(table).lower()
    for s in separators: # first split row into signs   
        row = row.replace(s, ' ').strip()
    s_l = row.split()
    s_l = [d2[sign] if sign in d2 else sign for sign in s_l]
    for sign in s_l:
        if sign[-1] == ')' and '(' in sign: # qualified sign - get only the qualifier
            sign = sign.split('(')[1][:-1]
        if '.' in sign or '+' in sign: 
            for s in separators2:
                sign = sign.replace(s, ' ').strip() 
            sign_l = sign.split()
            row_l.extend(sign_l)
            continue
        if '×' in sign and not '|' in sign:
            sign_l = sign.split('×')
            sign_l = [d2[sign] if sign in d2 else sign for sign in sign_l]
            sign = '|' + '×'.join(sign_l) + '|'
        row_l.append(sign)
        sign_names = [d2[sign] if sign in d2 else sign for sign in row_l]
    return ' '.join(sign_names).upper()

In [ ]:
df["sign_names"] = df["text"].progress_apply(signs)

# The Search Function
The search function takes as input any style of transliteration recognized in [OGSL](http://orac.org/ogsl) in upper or lower case. Signs may be connected with hyphens or spaces, determinatives may be written between curly brackets ({d}En-ki), or on the line (d-nin-gisz-zi-da). Shin may be represented by š, c, or sz and sign index numbers may be written on the line, or with Unicode subscript numbers ('e₂' and 'e2' are equivalent, but 'é' will yield no results). '{d}Nin-giš-zi-da-ke₄', 'd-nin-ŋeš-zi-da-ke₄', or 'AN nin gisz ZI da ke4' will all return the same results.  

The search engine will find any matching sequence of signs, independent of the transliteration, thus 'nig2 sig' will also find 'ninda sig'.

The search results are listed in a DataFrame. If there are 25 results or less, the DataFrame provides links to the [BDTNS](http://bdtns.filol.csic.es) pages of the matching texts.

In [ ]:
num = '0123456789x{}-c*'
ind = '₀₁₂₃₄₅₆₇₈₉ₓ   š×'
tab = str.maketrans(num, ind)
anchor = '<a href="http://bdtns.filol.csic.es/{}", target="_blank">{}</a>'

In [ ]:
def search(search): 
    search = search.lower().replace('sz', 'š').translate(tab).strip()
    search_l = search.split()
    search_l = [d2[s] if s in d2 else s for s in search_l]
    row_l = []
    for sign in search_l: 
        if '.' in sign or '+' in sign: 
            for s in separators2:
                sign = sign.replace(s, ' ').strip() 
                sign_l = sign.split()
            row_l.extend(sign_l)
        elif '×' in sign:
            sign_l = sign.replace('|', '').split('×')
            sign_l = [d2[sign] if sign in d2 else sign for sign in sign_l]
            sign = '|' + '×'.join(sign_l) + '|'
            row_l.append(sign)
        else: 
            row_l.append(sign)
        print(row_l)
    search_l = [re.escape(s) for s in row_l]
    signs = ' '.join(search_l).upper()
    show = ['id_text', 'line_label', 'text']
    #results = df[show].loc[df['sign_names'].str.contains('(?:(?<=\s)|(?<=^))'+signs+'(?=\s|$)', regex=True)].copy()
    results = df[show].loc[df['sign_names'].str.contains(r'\b'+signs+r'\b', regex=True)].copy()
    print(signs)
    print(str(len(results)) + ' hits')
    if len(results) <= 25: # add links only for 25 hits or less
        results['id_text'] = [anchor.format(val,val) for val in results['id_text']]
        results = results.style
    return results

# Search Instructions
Search for a sequence of sign values in any transliteration system recognized by [OGSL](http://oracc.org/ogsl). Thus, sugal₇, sukkal, or luh, in upper or lower case will all return the same results.

The Shin may be represented by š, c, or sz in upper or lower case.

Sign indexes may be represented by regular numbers or by index numbers (sig₇ or sig7).

Compound signs (such as diri) are resolved in their component signs if the compound represents a simple sequence of signs. Thus diri is resolved as SI A, but gu₇ is resolved as |KA×GAR|.

To search for a compound sign by sign name, enter it between pipes (|). The "times" sign may be represented by \* (enter |UR₂×A| or |UR₂\*A|).

In [ ]:
s = input()

In [ ]:
search(s)

In [ ]:
df[df['text'].str.contains('esir₂')]

In [ ]:
s in df.iloc[195]['sign_names']

In [ ]:
s

In [ ]:
df[df["text"].str.contains('diri')]

In [ ]:
df[df['sign_names'].str.contains('SI\.A')]

In [ ]:
df[df['sign_names'].str.contains('A₂ SAL.KUR KA', regex=False)]

In [ ]:
show = ['id_text', 'line_label', 'text']
df[show]

In [8]:
df

,id_text,id_line,line_label,text,comments
0,021035,1,o. 1,5 sila₃ kaš 3 sila₃ zi₃,
1,021035,2,o. 2,1 i₃ a₂-GAM,
2,021035,3,o. 3,Lu₂-Ma₂-gan-na lu₂-{giš}tukul-gu-<la>,
3,021035,4,o. 4,0.0.1 kaš 5 sila₃ zi₃,
4,021035,5,o. 5,1 i₃ a₂-GAM,
5,021035,6,o. 6,da-da sukkal ša₃ giš-/kin-ti-da gen-na,
6,021035,7,o. 8,3 sila₃ kaš 2 sila₃ zi₃,
7,021035,8,o. 9,1 i₃ a₂-GAM,
8,021035,9,o. 10,En-u₂-mi-i₃-li₂,
9,021035,10,o. 11,ma₂ giš-še₃ gen-na,


In [ ]:
%%timeit
search('diri-ga')

In [ ]:
d2['diri']